In [2]:
%pylab inline
from astropy.io import fits
from astropy.utils.data import get_pkg_data_filename
from astropy.table import Table
from astropy.wcs import WCS

Populating the interactive namespace from numpy and matplotlib


In [38]:
eazy = Table.read('boss1244_DR210308_zq_2to2.5_phot.fits')
eazy = eazy['id','f125w_flux_aper_3','f125w_fluxerr_aper_3','f160w_flux_aper_3','f160w_fluxerr_aper_3']

In [17]:
jh = fits.open('../LBC_CFHT_1244_1542/boss1244_DR210308_zq_2to2.5_phot.fits')

u1244 = fits.open('../LBC_CFHT_1244_1542/BOSS1244_U.cat.fits')
z1244 = fits.open('../LBC_CFHT_1244_1542/BOSS1244_z.cat.fits')
k1244 = loadtxt('../LBC_CFHT_1244_1542/j1244_emitter_50A_clean.cat')
pecat = loadtxt('../LBC_CFHT_1244_1542/zq_2to2.5.pecat')
pecat = pecat[argsort(pecat[:,0])]

u1244_data = u1244[2].data
z1244_data = z1244[2].data
ra1244 = u1244_data[:]['X_WORLD']
dec1244 = u1244_data[:]['Y_WORLD']

magu = u1244_data[:]['MAG_BEST']
magz = z1244_data[:]['MAG_BEST']
magu_err = u1244_data[:]['MAGERR_BEST']
magz_err = z1244_data[:]['MAGERR_BEST']
magk = k1244[:,4]
magk_err = k1244[:,5]
magh = k1244[:,6]
magh_err = k1244[:,7]


# fluxu = u1244_data[:]['FLUX_BEST']
# fluxz = z1244_data[:]['FLUX_BEST']
# fluxu_err = u1244_data[:]['FLUXERR_BEST']
# fluxz_err = z1244_data[:]['FLUXERR_BEST']
# fluxk = k1244[:]
# fluxk_err = k1244[:]

hst_125w = fits.open('../boss1244_mosaics_coadd/boss1244_DR210308-f125w_drz_sci.fits.gz')
hst_160w = fits.open('../boss1244_mosaics_coadd/boss1244_DR210308-f160w_drz_sci.fits.gz')

PHOTFLAM_125w = hst_125w[0].header['PHOTFLAM']
PHOTPLAM_125w = hst_125w[0].header['PHOTPLAM']

PHOTFLAM_160w = hst_160w[0].header['PHOTFLAM']
PHOTPLAM_160w = hst_160w[0].header['PHOTPLAM']

zp125 = -2.5*log10(PHOTFLAM_125w)-5*log10(PHOTPLAM_125w)-2.408
zp160 = -2.5*log10(PHOTFLAM_160w)-5*log10(PHOTPLAM_160w)-2.408

flux125 = eazy['f125w_flux_aper_3']
flux160 = eazy['f160w_flux_aper_3']

def mag2flux(mag,magerr):
    return 10**(-(mag+48.6)/2.5)/(1e-23)*1e6,abs(magerr*10**(-(mag+48.6)/2.5)*log(10)*(-1/2.5)/(1e-23)*1e6)

In [37]:
mag125 = -2.5*log10(flux125/0.1)+zp125
mag125_err = -2.5*log10(np.e)/mag125
mag160 = -2.5*log10(flux160/0.1)+zp160
mag160_err = -2.5*log10(np.e)/mag160

flux125w,flux125w_err= mag2flux(mag125,mag125_err)
flux160w,flux160w_err= mag2flux(mag160,mag160_err)

In [11]:
fluxu,fluxu_err= mag2flux(magu,magu_err)
fluxz,fluxz_err= mag2flux(magz,magz_err)
fluxk,fluxk_err= mag2flux(magk,magk_err)
fluxh,fluxh_err= mag2flux(magh,magh_err)

In [54]:
def sinhmag2flux(mag):
    return 2*1.4e-10*3767*np.sinh(mag/-2.5*log(10) - log(1.4e-10))

In [12]:
idx_no_uz  = []
idx_yes_orig1 = []
idx_yes_uz = []
for i,ra in enumerate(pecat[:,1]):
    idx = where((ra1244-ra)**2+(dec1244-pecat[i,2])**2==((ra1244-ra)**2+(dec1244-pecat[i,2])**2).min())[0][0]
    if (sqrt((ra1244-ra)**2+(dec1244-pecat[i,2])**2)).min()>0.5/3600:
        idx_no_uz.append(i)
    else:
        idx_yes_orig1.append(i)
        idx_yes_uz.append(idx)
        
fluxu_add = -99*ones(71)
fluxu_err_add = -99*ones(71)
fluxz_add = -99*ones(71)
fluxz_err_add = -99*ones(71)
fluxk_add = -99*ones(71)
fluxk_err_add = -99*ones(71)
fluxh_add = -99*ones(71)
fluxh_err_add = -99*ones(71)

fluxu_add[idx_yes_orig1] = fluxu[idx_yes_uz]
fluxu_err_add[idx_yes_orig1] = fluxu_err[idx_yes_uz]
fluxz_add[idx_yes_orig1] = fluxz[idx_yes_uz]
fluxz_err_add[idx_yes_orig1] = fluxz_err[idx_yes_uz]

idx_no_k = []
idx_yes_k = []
idx_yes_orig2 = []
for i,ra in enumerate(pecat[:,1]):
    idx = where((k1244[:,1]-ra)**2+(k1244[:,2]-pecat[i,2])**2==((k1244[:,1]-ra)**2+(k1244[:,2]-pecat[i,2])**2).min())[0][0]
    if (sqrt((k1244[:,1]-ra)**2+(k1244[:,2]-pecat[i,2])**2)).min()>0.5/3600:
        idx_no_k.append(i)
    else:
        idx_yes_k.append(idx)
        idx_yes_orig2.append(i)
fluxk_add[idx_yes_orig2] = fluxk[idx_yes_k]
fluxk_err_add[idx_yes_orig2] = fluxk_err[idx_yes_k]

fluxh_add[idx_yes_orig2] = fluxh[idx_yes_k]
fluxh_err_add[idx_yes_orig2] = fluxh_err[idx_yes_k]


In [39]:
eazy.replace_column(name = 'f125w_flux_aper_3',col=flux125w)
eazy.replace_column(name = 'f125w_fluxerr_aper_3',col=flux125w_err)

eazy.replace_column(name = 'f160w_flux_aper_3',col=flux160w)
eazy.replace_column(name = 'f160w_fluxerr_aper_3',col=flux160w_err)

eazy.add_column(name = 'flux_best_U',col=fluxu_add)
eazy.add_column(name = 'fluxerr_best_U',col=fluxu_err_add)

eazy.add_column(name = 'flux_best_z',col=fluxz_add)
eazy.add_column(name = 'fluxerr_best_z',col=fluxz_err_add)

eazy.add_column(name = 'flux_best_Ks',col=fluxk_add)
eazy.add_column(name = 'fluxerr_best_Ks',col=fluxk_err_add)

# eazy.add_column(name = 'flux_best_H2',col=fluxh_add)
# eazy.add_column(name = 'fluxerr_best_H2',col=fluxh_err_add)

eazy.add_column(name ='z_spec',col=pecat[:,-2])

In [41]:
eazy.write('/home/zihao/win/学习/天体物理/eazy-photoz-master/inputs/boss2.cat',format='ascii.commented_header',overwrite=True)

In [40]:
eazy

id,f125w_flux_aper_3,f125w_fluxerr_aper_3,f160w_flux_aper_3,f160w_fluxerr_aper_3,flux_best_U,fluxerr_best_U,flux_best_z,fluxerr_best_z,flux_best_Ks,fluxerr_best_Ks,z_spec
int32,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64
10,0.5622924626030109,0.02292722933862487,1.450459906993719,0.061731586576618175,0.061793550848960876,0.011667624115943909,0.31658628582954407,0.04719769209623337,-99.0,-99.0,2.0745
61,1.3712307696642738,0.058208505021434154,1.8344225291837328,0.07892959311355181,2.1614320278167725,0.05200577527284622,1.9544097185134888,0.07338385283946991,-99.0,-99.0,2.0575
119,0.25127540982881597,0.009892878716961755,0.5601217579665874,0.022834809694580446,-99.0,-99.0,-99.0,-99.0,-99.0,-99.0,2.0074
211,0.9208648670826276,0.03838614702318788,1.6360050662629197,0.07001786301672555,0.12103328108787537,0.015996668487787247,0.6298847794532776,0.06470329314470291,-99.0,-99.0,2.436
262,2.916086625557146,0.12824728474570207,7.600994341959509,0.3503113653268624,0.33733758330345154,0.022223200649023056,1.3031717538833618,0.08881016820669174,15.622840539708681,0.3352676119484324,2.2229
284,0.828668436586837,0.03437880792240428,1.7247785726455884,0.07399889489639422,0.8140448331832886,0.01874999701976776,0.8078052997589111,0.06623725593090057,-99.0,-99.0,2.0064
313,0.5293394042522122,0.021526030968291927,1.393728137108988,0.05920792245857245,0.15054941177368164,0.01321849413216114,0.3824283480644226,0.052004069089889526,1.74759180700884,0.1773769874195882,2.2273
381,0.7424469640901762,0.03065007030836506,1.2548538351430094,0.05305146915332327,0.23810896277427673,0.01848226599395275,0.48493149876594543,0.07303189486265182,-99.0,-99.0,2.179
382,0.595698144412204,0.024351547974127398,1.5784918089066238,0.06744424989349744,-99.0,-99.0,-99.0,-99.0,8.420153329554074,0.37845609336917724,2.2464


In [32]:
eazy

id,f125w_flux_aper_3,f125w_fluxerr_aper_3,f160w_flux_aper_3,f160w_fluxerr_aper_3,flux_best_U,fluxerr_best_U,flux_best_z,fluxerr_best_z,flux_best_Ks,fluxerr_best_Ks,z_spec
int32,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64
10,0.05622924626030109,0.002080630895942121,0.14504599069937188,0.0055794997589200465,0.061793550848960876,0.011667624115943909,0.31658628582954407,0.04719769209623337,-99.0,-99.0,2.0745
61,0.13712307696642737,0.0052623824755873684,0.1834422529183733,0.0071263921445881995,2.1614320278167725,0.05200577527284622,1.9544097185134888,0.07338385283946991,-99.0,-99.0,2.0575
119,0.025127540982881596,0.0009006408136435094,0.05601217579665874,0.0020722766943315064,-99.0,-99.0,-99.0,-99.0,-99.0,-99.0,2.0074
211,0.09208648670826276,0.0034763378361596566,0.16360050662629197,0.006325037745712209,0.12103328108787537,0.015996668487787247,0.6298847794532776,0.06470329314470291,-99.0,-99.0,2.436
262,0.2916086625557146,0.011554349580695815,0.7600994341959509,0.031411892033039235,0.33733758330345154,0.022223200649023056,1.3031717538833618,0.08881016820669174,15.622840539708681,0.3352676119484324,2.2229
284,0.08286684365868369,0.0031148208587174165,0.17247785726455886,0.006683072832739152,0.8140448331832886,0.01874999701976776,0.8078052997589111,0.06623725593090057,-99.0,-99.0,2.0064
313,0.052933940425221225,0.001953955043950239,0.1393728137108988,0.005352350092716073,0.15054941177368164,0.01321849413216114,0.3824283480644226,0.052004069089889526,1.74759180700884,0.1773769874195882,2.2273
381,0.07424469640901762,0.002778271966795919,0.12548538351430094,0.0047980309043114825,0.23810896277427673,0.01848226599395275,0.48493149876594543,0.07303189486265182,-99.0,-99.0,2.179
382,0.059569814441220406,0.002209363353386359,0.15784918089066236,0.006093529146594471,-99.0,-99.0,-99.0,-99.0,8.420153329554074,0.37845609336917724,2.2464


m = log10(flux)+ZPAB